In [4]:
# 引入必要的库
import numpy as np
import pandas as pd
import time
import copy


In [3]:
# 读取数据
def data_preprocessing():
    """读取数据，数据储存在Excel中，表名为分别为Processing Time与Machines Sequence，分别为工序时间和机器序列
    Returns:
        pt: 工序时间
        ms: 机器序列
        num_mc: 机器台数
        num_job: 工序数
        num_gene: 单条染色体中的基因数
        """
    
    pt_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Processing Time",index_col =[0])
    ms_tmp=pd.read_excel("JSP_dataset.xlsx",sheet_name="Machines Sequence",index_col =[0])

    dfshape=pt_tmp.shape # (工序数，机器台数)
    num_mc=dfshape[1] # 机器台数
    num_job=dfshape[0] # 工序数
    num_gene=num_mc*num_job # 单条染色体中的基因数

    pt = [list(map(int, pt_tmp.iloc[i])) for i in range(num_job)] # 工序时间
    ms = [list(map(int,ms_tmp.iloc[i])) for i in range(num_job)] # 机器序列
    return pt, ms, num_mc, num_job, num_gene

pt, ms, num_mc, num_job, num_gene = data_preprocessing()

In [33]:
# 设定初始参数
POPULATION_SIZE = 30 # 种群大小
CROSSOVER_RATE = 0.8 # 交叉概率
MUTATION_RATE = 0.2 # 变异概率
MUTATION_SELECTION_RATE = 0.2 # 变异基因数目选择率
NUM_MUTATION_JOBS = round(num_gene*MUTATION_SELECTION_RATE) # 变异基因数
NUM_ITERATION = 1000 # 迭代次数
SEED = 1            # 随机种子
ckeck_time = False # 是否计算运行时间

In [43]:
# 初始化种群
def initialize_population(population_size, num_gene, num_job, seed=None):
    """初始化种群
    Args:
        population_size: 种群大小
        num_gene: 每条染色体的基因数
        num_job: 工序数
    Returns:
        population_list: 种群列表
    >>> initialize_population(4, 6, 3, 1)
    [[2, 1, 1, 0, 0, 2],
     [1, 2, 0, 2, 0, 1],
     [2, 0, 2, 0, 1, 1],
     [0, 2, 1, 0, 2, 1]]
        """
    np.random.seed(seed) # 设置随机种子
    population_list=[] # 用于存储种群

    for i in range(population_size):
        nxm_random_num=list(np.random.permutation(num_gene)) # 生成一个随机序列，from 0 to num_gene-1   
        population_list.append(nxm_random_num) # 添加到种群中
        for j in range(num_gene):
            population_list[i][j]=population_list[i][j]%num_job # 生成随机基因序列
    return population_list

In [67]:
# 交叉
def crossover(parent_list, population_size, num_gene, crossover_rate, seed=None):
    """交叉操作，交叉点为两个随机点
    Args:
        parent_list: 父代种群列表
        population_size: 种群大小
        num_gene: 每条染色体的基因数
        crossover_rate: 交叉概率
    Returns:
        offspring_list: 子代种群列表
    >>> crossover([[2, 1, 1, 0, 0, 2],
                   [1, 2, 0, 2, 0, 1],
                   [2, 0, 2, 0, 1, 1],
                   [0, 2, 1, 0, 2, 1]], 3, 6, 0.8 , 1)
    [[2, 1, 2, 0, 0, 2],
     [1, 2, 0, 2, 0, 1],
     [2, 0, 1, 0, 1, 1],
     [0, 2, 1, 0, 2, 1]]
        """
    np.random.seed(seed) # 设置随机种子
    parent_list=copy.deepcopy(parent_list)      # 深拷贝父母染色体
    offspring_list=copy.deepcopy(parent_list)   # 深拷贝子代染色体
    S=list(np.random.permutation(population_size))  # 返回随机的序列，用于选择交配父母染色体
    
    for m in range(int(population_size/2)):         # 交配次数
        crossover_prob=np.random.rand()             # 生成随机交叉概率
        if crossover_rate>=crossover_prob:          # 交叉
            parent_1= parent_list[S[2*m]][:]    # 相邻两个进行交叉
            parent_2= parent_list[S[2*m+1]][:]
            child_1=parent_1[:]
            child_2=parent_2[:]
            cutpoint=list(np.random.choice(num_gene, 2, replace=False)) # 生成两个随机的交叉点
            cutpoint.sort() # 交叉点排序
        
            child_1[cutpoint[0]:cutpoint[1]]=parent_2[cutpoint[0]:cutpoint[1]]
            child_2[cutpoint[0]:cutpoint[1]]=parent_1[cutpoint[0]:cutpoint[1]]
            offspring_list[S[2*m]]=child_1[:]
            offspring_list[S[2*m+1]]=child_2[:]
    return offspring_list

In [71]:
def repair(population_size, offspring_list, num_job, num_mc):
    """修复错误的染色体，不符合工序流程
        分别记录每个机器上的工序，统计多余的与缺少的，将多余与缺少的互换
    Args:
        population_size: 种群大小
        offspring_list: 子代种群列表
        num_job: 工序数
        num_mc: 机器台数
    Returns:
        repaired_list: 修复后的子代种群列表
    >>> repair(4, [[2, 1, 2, 0, 0, 2],
                  [1, 2, 0, 2, 0, 1],
                  [2, 0, 1, 0, 1, 1],
                  [0, 2, 1, 0, 2, 1]], 3, 2)
    [[1, 1, 2, 0, 0, 2],
     [1, 2, 0, 2, 0, 1],
     [2, 0, 1, 0, 1, 1],
     [0, 2, 1, 0, 2, 1]]
        """
    for m in range(population_size):
        job_count={} # 用于记录每个数字出现的次数和第一次出现的位置
        larger,less=[],[] # 'larger'记录超过正常次数的染色体，less记录小于正常次数的染色体
        for i in range(num_job):
            if i in offspring_list[m]:
                count=offspring_list[m].count(i) # 计算染色体中每个数字出现的次数
                pos=offspring_list[m].index(i) # 记录第一次出现的位置
                job_count[i]=[count,pos] # 将以上两个值存储到job_count字典中
            else:
                count=0
                job_count[i]=[count,0]
            if count>num_mc:
                larger.append(i)
            elif count<num_mc:
                less.append(i)
                
        for k in range(len(larger)):
            chg_job=larger[k] # 需要处理的数字
            while job_count[chg_job][0]>num_mc:
                for d in range(len(less)):
                    if job_count[less[d]][0]<num_mc:                    
                        offspring_list[m][job_count[chg_job][1]]=less[d] # 将超过正常次数的数字替换为小于正常次数的数字
                        job_count[chg_job][1]=offspring_list[m].index(chg_job) # 更新第一次出现的位置
                        job_count[chg_job][0]=job_count[chg_job][0]-1 # 更新出现次数
                        job_count[less[d]][0]=job_count[less[d]][0]+1 # 更新出现次数                   
                    if job_count[chg_job][0]==num_mc: # 如果处理完毕，则跳出循环
                        break  
        repaired_list=offspring_list
        return repaired_list

In [92]:
# 变异
def mutate(offspring_list, num_gene, mutation_rate, num_mutation_jobs, seed = None):
    """变异操作
    Args:
        offspring_list: 子代种群列表
        num_gene: 每条染色体的基因数
        mutation_rate: 变异概率
        num_mutation_jobs: 每条染色体变异的基因数
    Returns:
        mutated_list: 变异后的子代种群列表
    >>> mutate([[1, 1, 2, 0, 0, 2],
                [1, 2, 0, 2, 0, 1],
                [2, 0, 1, 0, 1, 1],
                [0, 2, 1, 0, 2, 1]], 6, 1, 2, 2)"""
    [[1, 0, 2, 0, 1, 2],
     [1, 2, 0, 2, 0, 1],
     [2, 1, 1, 0, 1, 0],
     [2, 0, 1, 0, 2, 1]]
    np.random.seed(seed) # 设置随机种子
    for m in range(len(offspring_list)):
        mutation_prob=np.random.rand() # 生成随机变异概率
        if mutation_rate >= mutation_prob:
            m_chg=list(np.random.choice(num_gene, num_mutation_jobs, replace=False)) # 选择变异的位点
            t_value_last=offspring_list[m][m_chg[0]] # 记录第一个变异位点的值
            for i in range(num_mutation_jobs-1):
                offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] # 展示变异
            
            offspring_list[m][m_chg[num_mutation_jobs-1]]=t_value_last # 将第一个变异位点的值赋给最后一个变异位点
    mutated_list=offspring_list    
    return mutated_list

In [138]:
# 适应度计算
def calculate_fitness(population_size,total_chromosome, pt, ms, num_job, num_mc):
    """计算适应度
    Args:
        total_chromosome: 所有染色体
        pt: 加工时间矩阵
        ms: 移动时间矩阵
        num_job: 工序数
        num_mc: 机器台数
    Returns:
        chrom_fitness: 每条染色体的适应度
        chrom_fit: 每条染色体的适应度值
        total_fitness: 种群的适应度值
    >>> calculate_fitness(4,
                  [[2, 1, 1, 0, 0, 2],
                   [1, 2, 0, 2, 0, 1],
                   [2, 0, 2, 0, 1, 1],
                   [0, 2, 1, 0, 2, 1],
                   [1, 0, 2, 0, 1, 2],
                   [1, 2, 0, 2, 0, 1],
                   [2, 1, 1, 0, 1, 0],
                   [2, 0, 1, 0, 2, 1]], 
                  [[2, 3, 1], 
                   [2, 1, 2], 
                   [3, 2, 2]], 
                  [[1, 2, 3], 
                   [2, 2, 2], 
                   [1, 2, 1]], 3, 3)
    ([0.1,
      0.1111111111111111,
      0.09090909090909091,
      0.125,
      0.125,
      0.1111111111111111,
      0.125,
      0.09090909090909091],
     [10, 9, 11, 8, 8, 9, 8, 11],
      0.8790404040404042)
    """
    # total_chromosome=copy.deepcopy(parent_list)+copy.deepcopy(offspring_list) # 将亲代和子代染色体合并
    chrom_fitness,chrom_fit=[],[] # 用于存储每个染色体的倒数适应度（用于轮盘）和适应度
    total_fitness=0 # 总适应度
    for m in range(population_size*2):      # 计算每个染色体的适应度
        j_keys=[j for j in range(num_job)] 
        key_count={key:0 for key in j_keys} # 初始化字典，存储每个任务已经处理的次数
        j_count={key:0 for key in j_keys}   # 初始化字典，存储每个任务的累积处理时间
        m_keys=[j+1 for j in range(num_mc)]
        m_count={key:0 for key in m_keys}   # 初始化字典，存储每台机器的累积处理时间
        
        for i in total_chromosome[m]:       # 计算每个任务的累积处理时间
            gen_t=int(pt[i][key_count[i]])  # 获取当前任务的处理时间
            gen_m=int(ms[i][key_count[i]])  # 获取当前任务的处理机器
            j_count[i]=j_count[i]+gen_t     # 累积工序i的处理时间
            m_count[gen_m]=m_count[gen_m]+gen_t # 累积gen_m机器处理时间
            
            if m_count[gen_m]<j_count[i]:   # 如果机器累计处理时间小于工序处理时间，则将机器处理时间更新为工序处理时间
                m_count[gen_m]=j_count[i]
            elif m_count[gen_m]>j_count[i]: # 如果机器累计处理时间大于工序处理时间，则将工序处理时间更新为机器处理时间
                j_count[i]=m_count[gen_m]
                # 这样做是为了确保任务按顺序在机器上执行，即等待前一个任务完成后再开始执行下一个任务。
            
            key_count[i]=key_count[i]+1     # 更新任务处理次数
    
        makespan=max(j_count.values())      # 计算最大的累积处理时间，即最大的完成时间
        chrom_fitness.append(1/makespan)    # 计算每个染色体的适应度的倒数
        chrom_fit.append(makespan)          
        total_fitness=total_fitness+chrom_fitness[m] # 计算总适应度
    return chrom_fitness, chrom_fit, total_fitness

In [6]:
def selection(population_list,total_chromosome, chrom_fitness, total_fitness, population_size):
    """选择，轮盘赌，选择适应度高的染色体，生成新的种群
    Args:
        population_list: 种群
        total_chromosome: 所有染色体
        chrom_fitness: 每条染色体的适应度
        total_fitness: 种群的适应度值
        population_size: 种群大小
    Returns:
        new_population_list: 新的种群
    >>> selection(
                    [[2, 1, 1, 0, 0, 2],
                     [1, 2, 0, 2, 0, 1],
                     [2, 0, 2, 0, 1, 1],
                     [0, 2, 1, 0, 2, 1]],
                    [[2, 1, 1, 0, 0, 2],
                     [1, 2, 0, 2, 0, 1],
                     [2, 0, 2, 0, 1, 1], 
                     [0, 2, 1, 0, 2, 1],
                     [1, 0, 2, 0, 1, 2],
                     [1, 2, 0, 2, 0, 1],
                     [2, 1, 1, 0, 1, 0],
                     [2, 0, 1, 0, 2, 1]],
                    [0.1,
                     0.1111111111111111,
                     0.09090909090909091,
                     0.125,
                     0.125,
                     0.1111111111111111,
                     0.125,
                     0.09090909090909091],
                     0.8790404040404042, 8)
                    [[2, 1, 1, 0, 0, 2],
                     [1, 2, 0, 2, 0, 1],
                     [2, 0, 2, 0, 1, 1],
                     [0, 2, 1, 0, 2, 1],
                     [1, 0, 2, 0, 1, 2],
                     [1, 2, 0, 2, 0, 1],
                     [2, 1, 1, 0, 1, 0],
                     [2, 0, 1, 0, 2, 1]]
    [[2, 1, 1, 0, 1, 0],
     [0, 2, 1, 0, 2, 1],
     [0, 2, 1, 0, 2, 1],
     [0, 2, 1, 0, 2, 1]]
    """ 
    pk,qk=[],[] # 用于存储每个染色体的概率和累积概率
    
    for i in range(population_size*2):
        pk.append(chrom_fitness[i]/total_fitness)
    for i in range(population_size*2):
        cumulative=0
        for j in range(0,i+1):
            cumulative=cumulative+pk[j]
        qk.append(cumulative)
    
    selection_rand=[np.random.rand() for i in range(population_size)] # 生成随机数，用于选择
    
    for i in range(population_size):    # 根据随机数，选择染色体
        if selection_rand[i]<=qk[0]:    # 如果随机数小于第一个累积概率，则选择第一个染色体
            population_list[i]=copy.deepcopy(total_chromosome[0])
        else:
            for j in range(0,population_size*2-1):  # 如果随机数大于第一个累积概率，则选择第一个大于随机数的累积概率对应的染色体
                if selection_rand[i]>qk[j] and selection_rand[i]<=qk[j+1]:
                    population_list[i]=copy.deepcopy(total_chromosome[j+1])
                    break
    new_population_list=copy.deepcopy(population_list)
    return new_population_list

In [7]:
def crossover(population_list, population_size, chrom_fit, total_chromosome, crossover_rate):
    
    for i in range(population_size*2):
        if chrom_fit[i]<Tbest_now:
            Tbest_now=chrom_fit[i]
            sequence_now=copy.deepcopy(total_chromosome[i])
    if Tbest_now<=Tbest:
        Tbest=Tbest_now
        sequence_best=copy.deepcopy(sequence_now)
        
    makespan_record.append(Tbest)

[[2, 1, 1, 0, 1, 0],
 [0, 2, 1, 0, 2, 1],
 [0, 2, 1, 0, 2, 1],
 [0, 2, 1, 0, 2, 1]]